In [2]:
#import libraries

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from statistics import mean
from random import randint, uniform
from hyperopt import hp
from scipy.stats import randint, uniform


#import + split dataframe

df_train = pd.read_csv('train.csv', sep=',')
df_test = pd.read_csv('test-full.csv', sep=',')
x_train = df_train.drop(['Cover_Type'], axis=1)
y_train = df_train['Cover_Type'] - 1
x_test = pd.read_csv('test-full.csv', sep=',')

In [3]:
#training model

xgbc = xgb.XGBClassifier()
xgbc.fit(x_train, y_train)

#establishing baseline with no hyper-parameter optimisation or feature engineering

scores = cross_val_score(xgbc, x_train, y_train)
print(mean(scores))

0.8869708994708995


In [8]:
#random grid search CV
learning_rate = []
n_estimators = []
subsample = []
colsample_bytree = []
gamma = []
for i in range(100, 1000):
    learning_rate.append(uniform(0.01, 0.1))
    n_estimators.append(i)
    subsample.append(uniform(0.5, 0.5))
    colsample_bytree.append(uniform(0.5, 0.5))
    gamma.append(uniform(0, 0.5))

param_dist = {'max_depth' : [9, 10, 11, 12],
    'learning_rate' : uniform(0.01, 0.1),
    'n_estimators' : randint(100, 1000),
    'subsample' : uniform(0.5, 0.5),
    'colsample_bytree' : uniform(0, 0.5),
    'gamma' : uniform(0, 0.5)}

random_search = RandomizedSearchCV(xgbc, param_distributions = param_dist, n_iter = 10, cv=5, verbose=1, n_jobs=1)

random_search.fit(x_train, y_train)

print(random_search.best_params_)

best_xgbc = random_search.best_estimator_
scores = cross_val_score(best_xgbc, x_train, y_train)
print(mean(scores))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [56]:
print(random_search.best_params_)

{'colsample_bytree': 0.41938125697148076, 'gamma': 0.15632948894043341, 'learning_rate': 0.02465570471755834, 'max_depth': 10, 'n_estimators': 601, 'subsample': 0.8909036136371465}


In [88]:
y_pred = best_xgbc.predict(df_test)
df_test['Cover_Type'] = y_pred + 1
df_test

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


In [90]:
df_submission = pd.DataFrame({'Id': x_test['Id'].values, 'Cover_Type' : df_test['Cover_Type'].values})
df_submission['Cover_Type'].unique()

array([5, 2, 1, 7, 6, 3, 4], dtype=int64)

In [91]:
df_submission

,Id,Cover_Type
0,1,5
1,2,5
2,3,2
3,4,2
4,5,5
...,...,...
581007,581008,3
581008,581009,3
581009,581010,3
581010,581011,3


In [92]:
df_submission.to_csv('full_submission.csv', index = False)

In [93]:
submission_file = pd.read_csv('full_submission.csv')

In [80]:
submission_file

,Cover_Type,Id
0,5,1
1,5,2
2,2,3
3,5,4
4,5,5
...,...,...
581007,3,581008
581008,3,581009
581009,3,581010
581010,3,581011
